In [ ]:
%%sh
pip -q install pandas scikit-learn joblib

In [ ]:
import sagemaker

sess   = sagemaker.Session()
bucket = sess.default_bucket()                     
prefix = 'sklearn-boston-housing'

training = sess.upload_data(path='housing.csv', key_prefix=prefix + "/training")
output   = 's3://{}/{}/output/'.format(bucket,prefix)
print(training)
print(output)

In [ ]:
from sagemaker.sklearn import SKLearn

sk = SKLearn(entry_point='sklearn-boston-housing.py',
             role=sagemaker.get_execution_role(),
             framework_version='0.23-1',
             instance_count=1, 
             instance_type='ml.m5.large',
             output_path=output,
             hyperparameters={
                  'normalize': True,
                  'test-size': 0.1
              }
)

sk.fit({'training':training})

In [ ]:
%%sh -s $sk.model_data
aws s3 cp $1 .
tar xvfz model.tar.gz

In [ ]:
import joblib

model = joblib.load('model.joblib')
type(model)